In [1]:
import pandas as pd
import pymysql
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle

def obtener_datos_bbdd():
    """
    Conecta a la base de datos y extrae los datos necesarios desde precios_historicos.

    Retorno:
        pd.DataFrame: Un DataFrame con los datos de precio_apertura, precio_cierre, 
                      maximo, minimo y volumen.
    """
    try:
        # Configuración de conexión
        connection = pymysql.connect(
            host="pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com",
            user="admin",
            password="11jablum11",
            database="yfinance",
            port=3306
        )
        
        # Consulta para extraer los datos
        query = """
        SELECT precio_apertura, precio_cierre, maximo, minimo, volumen
        FROM precios_historicos
        """
        
        # Cargar los datos en un DataFrame
        df = pd.read_sql(query, connection)
        print("Datos cargados correctamente desde la base de datos.")
        return df

    except pymysql.MySQLError as e:
        print(f"Error al conectar a la base de datos: {e}")
        return pd.DataFrame()  # Retorna un DataFrame vacío en caso de error

    finally:
        if 'connection' in locals() and connection:
            connection.close()

# Obtener datos desde la base de datos
datos = obtener_datos_bbdd()

# Verificar si se obtuvieron datos
if datos.empty:
    print("No se obtuvieron datos desde la base de datos. Revisa la conexión o la consulta.")
else:
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test = train_test_split(datos, test_size=0.2, random_state=42)

    # Crear el escalador
    scaler = MinMaxScaler()

    # Ajustar el escalador solo con el conjunto de entrenamiento
    scaler.fit(X_train)

    # Escalar los datos
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convertir los datos escalados a DataFrames para mantener claridad
    columnas = ['precio_apertura', 'precio_cierre', 'maximo', 'minimo', 'volumen']
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=columnas, index=X_train.index)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=columnas, index=X_test.index)

    # Guardar el conjunto de entrenamiento escalado en un archivo .pkl
    with open("X_train_scaled.pkl", "wb") as archivo:
        pickle.dump(X_train_scaled, archivo)

    # Guardar el conjunto de prueba escalado en un archivo .pkl
    with open("X_test_scaled.pkl", "wb") as archivo:
        pickle.dump(X_test_scaled, archivo)

    print("Conjuntos de entrenamiento y prueba escalados guardados correctamente.")




/var/folders/3l/2mfq68kd2m7_kpj_3qkpl1qw0000gn/T/ipykernel_57254/301477760.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


Datos cargados correctamente desde la base de datos.
Conjuntos de entrenamiento y prueba escalados guardados correctamente.


In [3]:
from funciones import obtener_cotizaciones
cotizaciones_df = obtener_cotizaciones()

In [4]:
cotizaciones_df.head()

,Date,Close,Company
0,2000-01-03,19.8138,3M Company
1,2000-01-04,19.0265,3M Company
2,2000-01-05,19.5776,3M Company
3,2000-01-06,21.1523,3M Company
4,2000-01-07,21.5721,3M Company
